https://github.com/oreilly-japan/deep-learning-from-scratch-2/tree/master/ch01

In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [3]:
def linear_combination(w, x, b):
    return np.dot(x,w) +b

In [4]:
def curr_layer(w, input_layer, bias):
    not_activated = linear_combination(w, input_layer, bias)
    res = sigmoid(not_activated)
    return res

In [5]:
def derivative_sigmoid(output):
    return output*(1-output)

In [7]:
## todo: change the following part as a function
def forward_(input_, weight_layers, bias_layers):
    layers = [input_]
    for i in range(len(weight_layers)):
        weight, bias = weight_layers[i], bias_layers[i]
        prev_layer = layers[i]
        curr_ = curr_layer(weight, prev_layer, bias)
        layers += [curr_]
        
    #print([i.shape for i in layers])
    return layers

In [8]:
## derivative(C/w(L-1)) = d(z/w(L-1))*d(a/z)*d(C/a)
## derivative(C/a(L-1)) = d(z/a(L-1))*d(a/z)*d(C/a)

def derivative_all(layers, weight_layers, bias_layers, layer_no, ds):
    # layer_no:: lower layer: L-1 
    # return:: all variables: weight, bias and a
    weight, bias = weight_layers[layer_no], bias_layers[layer_no]
    layer, upper_ = layers[layer_no], layers[layer_no+1]
     
    ## weight update
    d_zw = layer
    d_az = derivative_sigmoid(upper_)
    d_cw = np.dot(d_zw.T, d_az*ds)
    updated_weight = weight - d_cw
    
    ## bias update
    d_cb = d_az*ds
    updated_bias = bias - d_cb
    
    ## activation update
    # is the updated weight or the original one?
    ds_lMinus1 = np.dot(d_az*ds, weight.T)
    
    return updated_weight, updated_bias, ds_lMinus1

In [9]:
# todo: add rounds 
## starting back-propagation

def backpropagation(layers, weight_layers, bias_layers, initial_d_ca):
    layer_len_ = len(layers)-1
    for i in reversed(range(layer_len_)):
        updated_weight, updated_bias, d_ca_lMinus1 = derivative_all(layers, weight_layers, bias_layers, i, initial_d_ca)
        weight_layers[i] = updated_weight
        bias_layers[i] = updated_bias
        initial_d_ca = d_ca_lMinus1
        
    return weight_layers, bias_layers

In [10]:
## initialzation
# 2 nodes input layer
# 3 nodes hidden layer
# 2 nodes output layers
input_, weight0, bias0 = np.random.random(20).reshape(10,2), np.random.random(6).reshape(2,3), np.random.random(3)
weight1, bias1 = np.random.random(6).reshape(3,2), np.random.random(2)

## the expected y
expected = np.random.randint(low=0, high = 2, size =2)
weight_layers = [weight0, weight1]
bias_layers = [bias0, bias1]

In [11]:
d_ca = float('inf')
while np.max(np.abs(d_ca))>0.001:
    layers = forward_(input_, weight_layers, bias_layers)
    output_layer = layers[-1]
    d_ca = output_layer - expected
    
    # re-calculate the weight and bias
    weight_layers, bias_layers = backpropagation(layers, weight_layers, bias_layers, d_ca)
    

In [ ]:
### comfirming 

from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=2, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
